In [1]:
from pathlib import Path
import tarfile
import os
import subprocess
import shlex
import json

from curriculum_deeplab.utils import get_script_dir
THIS_SCRIPT_DIR = get_script_dir()

In [5]:

# Download slic3er:
# https://download.slicer.org/

URL_SLICER_LINUX = "https://slicer-packages.kitware.com/api/v1/item/60add706ae4540bf6a89bf98/download"
tool_path = Path(THIS_SCRIPT_DIR, "tmp/tools")
slicer_tar_path = Path(tool_path, "slicer.tar.gz")
slicer_tar_path.parent.mkdir(parents=True, exist_ok=True)
!wget {URL_SLICER_LINUX} -O {slicer_tar_path}

In [7]:
# Download TCIA manifest file 
# https://wiki.cancerimagingarchive.net/pages/viewpage.action?pageId=70229053
tcia_data_path = Path(THIS_SCRIPT_DIR, "tmp/tcia_data")
tcia_file_path = tcia_data_path.joinpath("VS.tcia")
tcia_file_path.parent.mkdir(parents=True, exist_ok=True)
URL_TCIA = "https://wiki.cancerimagingarchive.net/download/attachments/70229053/Vestibular-Schwannoma-SEG%20Feb%202021%20manifest.tcia?api=v2"

!wget {URL_TCIA} -O {tcia_file_path}


--2022-04-01 14:12:08--  https://wiki.cancerimagingarchive.net/download/attachments/70229053/Vestibular-Schwannoma-SEG%20Feb%202021%20manifest.tcia?api=v2
Resolving wiki.cancerimagingarchive.net (wiki.cancerimagingarchive.net)... 144.30.169.13
Connecting to wiki.cancerimagingarchive.net (wiki.cancerimagingarchive.net)|144.30.169.13|:443... connected.
HTTP request sent, awaiting response... 200 
Length: 123472 (121K) [application/octet-stream]
Saving to: ‘/share/data_supergrover1/weihsbach/shared_data/tmp/curriculum_deeplab/curriculum_deeplab/preprocessing/tmp/tcia_data/VS.tcia’

/share/data_supergr 100%[===================>] 120.58K   286KB/s    in 0.4s    

2022-04-01 14:12:09 (286 KB/s) - ‘/share/data_supergrover1/weihsbach/shared_data/tmp/curriculum_deeplab/curriculum_deeplab/preprocessing/tmp/tcia_data/VS.tcia’ saved [123472/123472]



In [28]:
# https://wiki.cancerimagingarchive.net/display/Public/NBIA+Advanced+REST+API+Guide
import re
import zipfile
series_pattern = re.compile(r"^[0-9\.]+$")
def get_list_of_series(tcia_manifest_file_path):
    series_ids = []
    with open(tcia_manifest_file_path, 'r') as tcia_manifest_file:
        for line in tcia_manifest_file:
            for match in re.finditer(series_pattern, line.strip()):
                series_ids.append(match.group())
    return series_ids

def request_token(username, password, client_id, client_secret):
    request_token_address = f"'username={username}&password={password}&client_id={client_id}&client_secret={client_secret}&grant_type=password'"
    
    query_output = subprocess.check_output(shlex.split(f"curl -s -d {request_token_address} -X POST -k 'https://services.cancerimagingarchive.net/nbia-api/oauth/token'"))
    try:
        token = json.loads(query_output)['access_token']
    except (json.JSONDecodeError, KeyError):
        raise ValueError("TCIA token could not be fetched. Please check your credentials.")

    return token

def get_metadata(token, series_id):
    rest_download_address = f"https://services.cancerimagingarchive.net/nbia-api/services/v1/getSeriesMetaData?SeriesInstanceUID={series_id}"
    authorization_request = f'"Authorization:Bearer {token}"'
    query_output = subprocess.check_output(shlex.split(f"curl -s -k {rest_download_address}"))

    try:
        metadata = json.loads(query_output)[0]
    except (json.JSONDecodeError):
        return None

    return metadata
    # !curl -k {rest_download_address} --output {output_path}

def download_restricted_data(token, series_id, output_file_path):
    rest_download_address = f"https://services.cancerimagingarchive.net/nbia-api/services/v1/getImage?SeriesInstanceUID={series_id}"
    output_file_path.parent.mkdir(parents=True, exist_ok=True)
    authorization_request = f'"Authorization:Bearer {token}"'
    !curl -H {authorization_request} -k {rest_download_address} --output {output_file_path}

# https://wiki.cancerimagingarchive.net/plugins/personalaccesstokens/usertokens.action
all_series_ids = [get_list_of_series(tcia_file_path)[0]]
# all_series_ids = ["1.3.6.1.4.1.9590.100.1.2.374115997511889073021386151921807063992"]

token = request_token(input("Enter TCIA username"), input("Enter TCIA password"), input("Enter TCIA client_id from helpdesk"), input("Enter TCIA client_secret from helpdesk"))

for sid in all_series_ids:
    metadata = get_metadata(token, sid)
    if not metadata: continue

    zip_data_path = tcia_data_path.joinpath(metadata['Subject ID'] + ".zip")
    try:
        with zipfile.ZipFile(zip_data_path) as zf:
            pass
    except (FileNotFoundError, zipfile.BadZipfile):
        download_restricted_data(token, sid, zip_data_path)
    

ValueError: TCIA token could not be fetched. Please check your credentials.

In [51]:
!curl -d "username=nbia_guest" -X POST -k https://services.cancerimagingarchive.net/nbia-api/oauth/token

{"error":"unauthorized","error_description":"An Authentication object was not found in the SecurityContext"}

In [10]:
with tarfile.open(slicer_tar_path, 'r') as tar_file:
    tar_file.extractall(path=tool_dir)

In [2]:
tool_dir = Path(THIS_SCRIPT_DIR, "tmp/tools")

slicer_data_dir = [_dir for _dir in tool_dir.iterdir() if _dir.is_dir()][0]
slicer_bin = slicer_data_dir.joinpath("Slicer")
# slicer_python_bin = slicer_data_dir.joinpath("bin/PythonSlicer")

slicer_script = Path(THIS_SCRIPT_DIR, "data_conversion.py")

In [11]:
XVFB_LIB_PATH = "/share/data_supergrover1/weihsbach/miniconda3/envs/misc/lib"
XVFB_PATH = "/share/data_supergrover1/weihsbach/miniconda3/envs/misc/x86_64-conda_cos6-linux-gnu/sysroot/usr/bin/xvfb-run"
XVFB_BASE_PATH = "/share/data_supergrover1/weihsbach/miniconda3/envs/misc/x86_64-conda_cos6-linux-gnu/sysroot/usr/bin"
# os.environ['PATH'] = os.environ['PATH']+":"+XVFB_BASE_PATH
# os.environ['LD_LIBRARY_PATH'] = os.environ['LD_LIBRARY_PATH']+":"+XVFB_LIB_PATH

! {XVFB_PATH} -e xvfb.log {slicer_bin}  #-v --python-script {slicer_script}

Slicer: cannot connect to X server :99
/share/data_supergrover1/weihsbach/miniconda3/envs/misc/x86_64-conda_cos6-linux-gnu/sysroot/usr/bin/xvfb-run: 186: kill: No such process



In [53]:
!{XVFB_PATH} --help

Usage: xvfb-run [OPTION ...] COMMAND
Run COMMAND (usually an X client) in a virtual X server environment.
Options:
-a        --auto-servernum          try to get a free server number, starting at
                                    --server-num (deprecated, use --auto-display
                                    instead)
-d        --auto-display            use the X server to find a display number
                                    automatically
-e FILE   --error-file=FILE         file used to store xauth errors and Xvfb
                                    output (default: /dev/null)
-f FILE   --auth-file=FILE          file used to store auth cookie
                                    (default: ./.Xauthority)
-h        --help                    display this usage message and exit
-n NUM    --server-num=NUM          server number to use (default: 99)
-l        --listen-tcp              enable TCP port listening in the X server
-p PROTO  --xauth-protocol=PROTO    X authority protocol nam